In [11]:
!pip install -q geemap earthengine-api

In [12]:
# Set up EE
import ee
import geemap
ee.Authenticate()
ee.Initialize(project = "ee-frankfurt-fun-challenge")

In [13]:
# Get feature collection for German states
fc = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM0_NAME", "Germany"))

In [14]:
vis_params = {
    "color": "red",
    "fillColor": "00000000",
    "width": 3
}

Map = geemap.Map()
Map.centerObject(fc, 8)
Map.addLayer(fc.style(**vis_params), {}, "German States")
Map

Map(center=[51.05286656613144, 10.372108688819564], controls=(WidgetControl(options=['position', 'transparent_…

### Task 1: Locate Frankfurt/Main in Google Earth Engine and download a representative image of it

In [34]:
# Longitude and latitude of Franfurt
frankfurt_coords = [8.601211, 50.099498]
frankfurt = ee.Geometry.Point(frankfurt_coords)
frankfurt_square_region = frankfurt.buffer(10000).bounds()  # 10 km buffer for half side, then get bounds

In [35]:
# Load Sentinel-2 image collection, select Frankfurt and the desired dates, and choose the least cloudy image
collection = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
  .filterBounds(frankfurt_square_region) \
  .filterDate("2023-01-01", "2023-12-31") \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)) \
  .sort("CLOUDY_PIXEL_PERCENTAGE") \
  .first()

In [40]:
vis_params_frankfurt = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000
}
map_frankfurt = geemap.Map(center = frankfurt_coords[::-1], zoom = 12)
map_frankfurt.clear()
map_frankfurt.addLayer(collection.clip(frankfurt_square_region), vis_params_frankfurt, "Sentinel-2-Frankfurt")
map_frankfurt.addLayer(frankfurt_square_region, {}, "Frankfurt")
map_frankfurt

Map(center=[50.099498, 8.601211], crs={'name': 'EPSG3857', 'custom': False}, default_style=MapStyle(), draggin…